<a href="https://colab.research.google.com/github/adityakulkarni0910/Load_Forecast_Model/blob/main/Load_Forecast_Model_NYC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NYISO - Load data

In [2]:
from bs4 import BeautifulSoup
import requests

html_page = requests.get('http://mis.nyiso.com/public/P-58Blist.htm')
soup = BeautifulSoup(html_page.content, 'html.parser')

# find all a href links available on this page
file_urls = soup.findAll('a')
links = [link.get('href') for link in file_urls]
filtered_links = [link for link in links if link.endswith('.zip')]

In [3]:
#filtered_links
common_url = 'http://mis.nyiso.com/public/'
filtered_links = [common_url + s for s in filtered_links]

In [5]:

import pandas as pd
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

# initiating a dataframe
load_full = pd.DataFrame()

for link in filtered_links:
    # open url
    resp = urlopen(link)
    # read zipfile
    zipfile = ZipFile(BytesIO(resp.read()))
    # get the csv file names
    fname = zipfile.namelist()
    
    
    #for all files in every zip folder
    for file in fname:
      # convert to pandas dateframe
      load = pd.read_csv(zipfile.open(file) )
      
      #filtering data to NYC
      load = load[load['Name']=='N.Y.C.']

      # concatenate dataframes
      load_full = pd.concat([load_full,load], ignore_index=True, sort=False)
        
    # close zipfile we don't need
    zipfile.close()

In [6]:
load_full.head()

,Time Stamp,Time Zone,Name,PTID,Load
0,05/01/2021 00:00:00,EDT,N.Y.C.,61761.0,4261.2480
1,05/01/2021 00:05:00,EDT,N.Y.C.,61761.0,4252.7850
2,05/01/2021 00:10:00,EDT,N.Y.C.,61761.0,4233.1177
3,05/01/2021 00:15:00,EDT,N.Y.C.,61761.0,4213.7046
4,05/01/2021 00:20:00,EDT,N.Y.C.,61761.0,4179.1064


In [7]:
load_full['Time Stamp']=pd.to_datetime(load_full['Time Stamp'])

In [8]:
load_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1718386 entries, 0 to 1718385
Data columns (total 5 columns):
 #   Column      Dtype         
---  ------      -----         
 0   Time Stamp  datetime64[ns]
 1   Time Zone   object        
 2   Name        object        
 3   PTID        float64       
 4   Load        float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 65.6+ MB


In [9]:
load_h = load_full.set_index('Time Stamp').resample('60min').mean()
load_h = load_h.reset_index()
load_h.drop(columns=['PTID'],inplace=True)

In [10]:
load_h.head()

,Time Stamp,Load
0,2005-01-31 00:00:00,4616.341667
1,2005-01-31 01:00:00,4423.291667
2,2005-01-31 02:00:00,4297.016667
3,2005-01-31 03:00:00,4253.200000
4,2005-01-31 04:00:00,4284.258333


In [11]:
load_d = load_full.set_index('Time Stamp').resample('D').mean()
load_d = load_d.reset_index()
load_d.drop(columns=['PTID'],inplace=True)

In [12]:
load_d.head()

,Time Stamp,Load
0,2005-01-31,5657.095205
1,2005-02-01,5580.662630
2,2005-02-02,5535.835517
3,2005-02-03,5514.954639
4,2005-02-04,5717.773469


# NOAA - Weather data

In [30]:
#needed to make web requests
import requests

#store the data we get as a dataframe
import pandas as pd

#convert the response as a strcuctured json
import json

#mathematical operations on lists
import numpy as np

#parse the datetimes we get from NOAA
from datetime import datetime

#add the access token you got from NOAA
Token = 'ZhWNXHpCRhUObdcNxmSlkSXEPrCreqbc'

#Long Beach Airport station
station_id = 'GHCND:USW00094728'

In [31]:
from datetime import date

dataset_id = 'GHCND'

datatype_id = ['AWND', 'PRCP', 'SNOW', 'TAVG', 'TMAX',
       'TMIN', 'WDF2', 'WSF2', 'TSUN', 'WT01', 'WT08',
       'WT13', 'WT16', 'WT18', 'WT19', 'WT02', 'WT22']

limit = str(1000)

start_year = 2001

end_year = date.today().year

In [ ]:
#2

weather = pd.DataFrame()

#for each year from 2001-2021 ...
for year in range(start_year, end_year+1):
    year = str(year)
    
    for dtype in datatype_id:

    #make the api call
      r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid='+dataset_id+'&datatypeid='+dtype+'&limit='+limit+'&stationid='+station_id+'&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})

      d = json.loads(r.text)

      if d != {}:

        df = pd.DataFrame(d['results'])
        weather = pd.concat([weather, df], ignore_index=True, sort=False)

      else:
        print('year '+year+'- datatype '+dtype+' not available!')
    print('year '+year+' done')


year 2001 done
year 2002 done
year 2003 done
year 2004 done
year 2005 done
year 2006- datatype TAVG not available!
year 2006- datatype TSUN not available!
year 2006 done
year 2007- datatype TAVG not available!
year 2007- datatype TSUN not available!
year 2007 done
year 2008- datatype TAVG not available!
year 2008- datatype TSUN not available!
year 2008 done
year 2009- datatype TAVG not available!
year 2009- datatype TSUN not available!
year 2009 done
year 2010- datatype TAVG not available!
year 2010- datatype TSUN not available!
year 2010 done
year 2011- datatype TAVG not available!
year 2011- datatype TSUN not available!
year 2011 done
year 2012- datatype TAVG not available!
year 2012- datatype TSUN not available!
year 2012 done
year 2013- datatype TAVG not available!
year 2013- datatype TSUN not available!
year 2013 done
year 2014- datatype TAVG not available!
year 2014- datatype TSUN not available!
year 2014- datatype WT13 not available!
year 2014- datatype WT16 not available!
year 

In [ ]:
weather_2 = weather.pivot(index='date',columns='datatype',values='value')
weather_2.reset_index(inplace=True)
weather_2['date'] = pd.to_datetime(weather_2['date'])

In [ ]:
weather_2.head()

In [ ]:
weather_2.isnull().mean()

In [ ]:
weather_2.drop(columns=['TAVG', 'TSUN'],inplace=True)

In [ ]:
for i in weather_2.columns:
  if i.startswith('WT'):
#    print(i)
    weather_2[i].fillna(0,inplace=True)

In [ ]:
weather_2

# Pre-Processing

In [ ]:
load_d.rename(columns={'Time Stamp':'date'},inplace=True)
load_d.head()

,date,Load
0,2005-01-31,5657.095205
1,2005-02-01,5580.662630
2,2005-02-02,5535.835517
3,2005-02-03,5514.954639
4,2005-02-04,5717.773469


In [ ]:
weather_2.head()

datatype,date,AWND,PRCP,SNOW,TMAX,TMIN,WDF2,WSF2,WT01,WT02,WT08,WT13,WT16,WT18,WT19,WT22
0,2001-01-01,58.0,0.0,0.0,0.0,-50.0,0.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2001-01-02,46.0,0.0,0.0,-28.0,-72.0,320.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2001-01-03,28.0,0.0,0.0,-6.0,-78.0,270.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2001-01-04,30.0,0.0,0.0,11.0,-39.0,300.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2001-01-05,23.0,48.0,46.0,6.0,-44.0,270.0,67.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [ ]:
data = pd.merge(load_d, weather_2,how='left',on='date')
data.head()

,date,Load,AWND,PRCP,SNOW,TMAX,TMIN,WDF2,WSF2,WT01,WT02,WT08,WT13,WT16,WT18,WT19,WT22
0,2005-01-31,5657.095205,28.0,0.0,0.0,17.0,-44.0,20.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2005-02-01,5580.662630,12.0,0.0,0.0,44.0,-39.0,350.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2005-02-02,5535.835517,23.0,0.0,0.0,44.0,-22.0,60.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2005-02-03,5514.954639,34.0,5.0,0.0,50.0,-17.0,70.0,76.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2005-02-04,5717.773469,30.0,69.0,0.0,78.0,11.0,40.0,67.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [ ]:
data.isnull().mean()

date    0.000000
Load    0.005220
AWND    0.039064
PRCP    0.000505
SNOW    0.000505
TMAX    0.000505
TMIN    0.000505
WDF2    0.040916
WSF2    0.040411
WT01    0.000505
WT02    0.000505
WT08    0.000505
WT13    0.000505
WT16    0.000505
WT18    0.000505
WT19    0.000505
WT22    0.000505
dtype: float64

In [ ]:
data['Load'].interpolate(method='linear',inplace=True)
data.isnull().mean()

date    0.000000
Load    0.000000
AWND    0.039064
PRCP    0.000505
SNOW    0.000505
TMAX    0.000505
TMIN    0.000505
WDF2    0.040916
WSF2    0.040411
WT01    0.000505
WT02    0.000505
WT08    0.000505
WT13    0.000505
WT16    0.000505
WT18    0.000505
WT19    0.000505
WT22    0.000505
dtype: float64

In [ ]:
data.describe()

,Load,AWND,PRCP,SNOW,TMAX,TMIN,WDF2,WSF2,WT01,WT02,WT08,WT13,WT16,WT18,WT19,WT22
count,5939.000000,5707.000000,5936.000000,5936.000000,5936.000000,5936.000000,5696.000000,5699.000000,5936.000000,5936.000000,5936.000000,5936.000000,5936.000000,5936.000000,5936.000000,5936.000000
mean,6052.024384,24.754862,36.242419,2.297507,172.541274,93.586422,195.793188,64.399544,0.357817,0.033861,0.189522,0.174360,0.186489,0.032345,0.014319,0.006402
std,982.326552,11.172155,101.472963,21.027421,100.508320,91.709537,100.012640,18.204241,0.479398,0.180887,0.391955,0.379451,0.389534,0.176930,0.118814,0.079760
min,3662.738542,0.000000,0.000000,0.000000,-105.000000,-182.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5440.868621,17.000000,0.000000,0.000000,89.000000,22.000000,80.000000,54.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5793.049653,23.000000,0.000000,0.000000,178.000000,94.000000,230.000000,63.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,6414.391840,31.000000,15.000000,0.000000,261.000000,178.000000,280.000000,72.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,10163.128819,102.000000,1923.000000,693.000000,400.000000,289.000000,360.000000,170.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
import datetime as dt

data['date'] = data['date'].map(dt.datetime.toordinal)

In [ ]:
from sklearn.model_selection import train_test_split

X_trainval, X_test, y_trainval, y_test = train_test_split(data.drop('Load',axis=1), data.Load, test_size=0.2, random_state=0)

X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, random_state=0)

In [ ]:
print(f'Train size is {X_train.shape}' )
print(f'Test size is {X_test.shape}' )
print(f'Val size is {X_val.shape}' )

Train size is (3563, 16)
Test size is (1188, 16)
Val size is (1188, 16)


In [ ]:
!pip install feature-engine

In [ ]:
from feature_engine.imputation import MeanMedianImputer as mdi 
from feature_engine.imputation import CategoricalImputer, RandomSampleImputer

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, OneHotEncoder

from sklearn.model_selection import GridSearchCV, cross_val_score


In [ ]:
load_pipeline = Pipeline([
    
    # missing data imputation 
    ('imputer_num',
    mdi(variables=['AWND'	,'PRCP','SNOW',	'TMAX',	'TMIN',	'WDF2',	'WSF2'])),    
  
    ('imputer_cat',
     RandomSampleImputer(variables=['WT01' , 'WT02' , 'WT08' , 'WT13' , 'WT16' , 'WT18' , 'WT19' , 'WT22'])),           
     
    
    #categorical encoding
  

    #numerical encoding       
    
    # feature Scaling
    
 #  ('scaler', StandardScaler()),
 


])

In [ ]:
load_pipeline.fit(X_train)

Pipeline(memory=None,
         steps=[('imputer_num',
                 MeanMedianImputer(imputation_method='median',
                                   variables=['AWND', 'PRCP', 'SNOW', 'TMAX',
                                              'TMIN', 'WDF2', 'WSF2'])),
                ('imputer_cat',
                 RandomSampleImputer(random_state=None, seed='general',
                                     seeding_method='add',
                                     variables=['WT01', 'WT02', 'WT08', 'WT13',
                                                'WT16', 'WT18', 'WT19',
                                                'WT22']))],
         verbose=False)

In [ ]:
X_train=pd.DataFrame(load_pipeline.transform(X_train),columns=X_train.columns)
X_test=pd.DataFrame(load_pipeline.transform(X_test),columns=X_test.columns)
X_val=pd.DataFrame(load_pipeline.transform(X_val),columns=X_val.columns)

# Regression models

In [ ]:
#Linear regression

from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(X_train, y_train)

scores = cross_val_score(lr, X_train, y_train)

print("LR Cross-validation scores: {}".format(scores))
print("LR Average cross-validation score: {:.4f}".format(scores.mean()))
print("LR train performance:", lr.score(X_train,y_train))
print("LR test performance:", lr.score(X_test,y_test))

LR Cross-validation scores: [0.4071612  0.41548503 0.39298808 0.35678866 0.40412788]
LR Average cross-validation score: 0.3953
LR train performance: 0.4007161052587508
LR test performance: 0.3615391303134853


In [ ]:
# Ridge regression model
from sklearn.linear_model import Ridge
ridge = Ridge()
#List of parameters
param_ridge = {'alpha':[0.001, 0.01, 0.1, 1, 10, 100,1000]}
grid_ridge = GridSearchCV(ridge, param_ridge, cv=10, return_train_score = True)
grid_ridge.fit(X_train, y_train)
# Mean Cross Validation Score
print("Best Mean Cross-validation score: {:.3f}".format(grid_ridge.best_score_))
print()
#best parameters
print('Ridge parameters: ', grid_ridge.best_params_)
# Check train and test data set performance
print("Ridge Train Performance: ", grid_ridge.score(X_train, y_train))
print("Ridge Test Performance: ", grid_ridge.score(X_test,y_test))


Best Mean Cross-validation score: 0.386

Ridge parameters:  {'alpha': 1}
Ridge Train Performance:  0.40071179375467925
Ridge Test Performance:  0.3615746207941982


In [ ]:
from xgboost import XGBClassifier


In [ ]:
from  xgboost import XGBClassifier
xgbc= XGBClassifier(random_state=42,early_stopping_rounds=2,objective= 'binary:logistic')
xgbc_param = {
              'max_depth' : [2,4,6],
              'n_estimators' : [50,100,150],
              'learning_rate' : [0.1,0.5,0.8],
               'min_child_weight' : [1,3,5,7],
                'subsample':[0.6,0.7,0.8,0.9,1]
             }
xgbc_grid = GridSearchCV(xgbc, xgbc_param,cv=5, return_train_score=True,scoring='roc_auc' )
xgbc_grid.fit(X_train,y_train)

KeyboardInterrupt: ignored

In [ ]:
#Lasso regression model
from sklearn.linear_model import Lasso
13
lasso = Lasso(random_state=0,max_iter=10000)
#define a list of parameters
param_lasso = {'alpha':[0.001, 0.01, 0.1, 1, 10, 100] }
grid_lasso = GridSearchCV(lasso, param_lasso, cv=10, return_train_score = True)
grid_lasso.fit(X_train, y_train)
# Mean Cross Validation Score
print("Best Mean Cross-validation score: {:.3f}".format(grid_lasso.best_score_))
print()
# best parameters
print('Lasso parameters: ', grid_lasso.best_params_)
# Check test data set performance
print('Train score: ', grid_lasso.score(X_train, y_train))
print("Lasso Test Performance: ", grid_lasso.score(X_test,y_test))

Best Mean Cross-validation score: 0.386

Lasso parameters:  {'alpha': 0.001}
Train score:  0.40071610462769675
Lasso Test Performance:  0.36153951396754946


In [ ]:
# Decision Trees Regressor
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor()
param_grid = {'max_depth': range(1,10),
'min_samples_split': range(2,10,1)}
grid_tree = GridSearchCV(tree, param_grid, cv=5)
grid_tree.fit(X_train,y_train)
print("Best cross-validation score: {:.2f}".format(grid_tree.best_score_))
print(grid_tree.best_params_)
print('train score: ', grid_tree.score(X_train, y_train))
print('test score: ', grid_tree.score(X_test, y_test))

Best cross-validation score: 0.79
{'max_depth': 6, 'min_samples_split': 6}
train score:  0.8368676687573738
test score:  0.7979066173612149


In [ ]:
file = pd.merge(load_d, weather_2,how='left',on='date')
file.to_csv('data.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5939 entries, 0 to 5938
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    5939 non-null   datetime64[ns]
 1   Load    5908 non-null   float64       
 2   AWND    5707 non-null   float64       
 3   PRCP    5936 non-null   float64       
 4   SNOW    5936 non-null   float64       
 5   TMAX    5936 non-null   float64       
 6   TMIN    5936 non-null   float64       
 7   WDF2    5696 non-null   float64       
 8   WSF2    5699 non-null   float64       
 9   WT01    5936 non-null   float64       
 10  WT02    5936 non-null   float64       
 11  WT08    5936 non-null   float64       
 12  WT13    5936 non-null   float64       
 13  WT16    5936 non-null   float64       
 14  WT18    5936 non-null   float64       
 15  WT19    5936 non-null   float64       
 16  WT22    5936 non-null   float64       
dtypes: datetime64[ns](1), float64(16)
memory usage: 835.

The current weather normalized load forecasting model has an approximate best of 80%